In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
1380,1.0,0.0,10.258376,0.120406,0.845004,-1.380302,-2.834344,-8.844424
8625,1.0,1.0,-10.810000,-0.069796,-1.341209,0.577876,0.857943,0.868266
2632,1.0,0.0,-1.475738,-0.201067,-4.672531,-1.401264,-4.341143,-9.932812
5494,0.0,0.0,14.382996,0.093171,0.266830,0.869266,6.816091,-3.633813
4935,1.0,0.0,-6.948821,-0.274633,-1.303654,-1.724862,-3.422504,-4.604489
...,...,...,...,...,...,...,...,...
3682,1.0,1.0,-15.071200,-0.093570,-3.140276,1.438176,-3.144795,0.751739
6866,0.0,0.0,-4.389505,0.029832,0.646589,-1.793995,0.784591,-1.217587
1948,0.0,0.0,-2.884948,-0.160470,2.018382,-0.338872,-0.638602,1.572318
1065,1.0,1.0,-19.077107,-0.107236,-2.327832,-0.935472,-2.421513,-6.866882


In [3]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])
for name in os.listdir('./Model/'):
    full_name=name
    model=torch.load(f'./Model/{name}')
    name=name.split('_')[1]
    name=name.replace('.pth','')
    #print('FIRST',name)
    #print(name[:-1])
    if name[-1].isdigit():
        name=name[:-1]
    #print('Second',name)

    dataset = pd.read_csv(f'../data/{name}/{name}.csv')
    continuous_wachter = dataset.drop(columns=['label']).columns
    scaler='Identity'
    if name == 'economic':
        scaler='MinMax'

    dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
    dataset=dataset._df_test
    data_x, data_y=dataset.drop(columns=['label']), dataset['label']
    data_pred= model.predict(data_x)
    model_name.append(full_name)
    print(data_pred.shape)
    print(data_y.shape)
    if len(data_pred.shape)>1:
        data_pred= np.argmax(data_pred, axis=1)

    accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
    precision.append(sklearn.metrics.precision_score(data_y,data_pred))
    recall.append(sklearn.metrics.recall_score(data_y,data_pred))
    auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defin

,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.8092,0.831535,0.785266,0.809708
2,Forest_economic.pth,0.8032,0.755034,0.818182,0.804805
3,Forest_sanity-3-lin.pth,0.806,0.824291,0.791602,0.806427
4,Forest_sanity-3-non-add.pth,0.8624,0.950262,0.527616,0.758565
5,Forest_sanity-3-non-lin.pth,0.9628,0.974576,0.948063,0.962376
6,Linear_credit.pth,0.744,0.756442,0.723946,0.744113
7,Linear_economic.pth,0.3584,0.34282,0.474061,0.369447
8,Linear_sanity-3-lin.pth,0.994,0.993725,0.994505,0.99399
9,Linear_sanity-3-non-add.pth,0.9992,1,0.996942,0.998471


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    if st is '':
        st += row['model_name'].split('_')[1].replace('.pth','')
    else:
        st += ' & '+row['model_name'].split('_')[1].replace('.pth','')
    col.append(row['model_name'].split('_')[0])
    name.append(row['model_name'].split('_')[1])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name)
print(col)
print(name)
for i in range(0,len(col)):
    st += col[i] 
    for j in range(0,len(name)):

        #print()
        print(f'{col[i]}_{name[i]}')
        print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

['Forest' 'Linear' 'MLP']
['credit.pth' 'economic.pth' 'sanity-3-lin.pth' 'sanity-3-lin0.pth'
 'sanity-3-lin1.pth' 'sanity-3-lin2.pth' 'sanity-3-non-add.pth'
 'sanity-3-non-lin.pth']
Forest_credit.pth
0.8092
Forest_credit.pth
0.8032
Forest_credit.pth
0.806
Forest_credit.pth


IndexError: index 0 is out of bounds for axis 0 with size 0